In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv('nyra_2019_complete.csv', low_memory=False, parse_dates=['race_date'])
df2 = pd.read_csv('horse_ids.csv',parse_dates=['race_date'])
df3 = pd.read_csv('horse_names.csv')

In [3]:
df1['program_number'] = df1['program_number'].str.strip()
df2.drop('Unnamed: 0', axis=1, inplace=True)
df2.rename(columns={'race': 'race_number'}, inplace=True)
df3.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
df4 = pd.merge(df1,df2, on=['track_id','race_date','race_number','program_number'])

In [5]:
df5 = pd.merge(df4,df3, on=['horse_id'])

In [6]:
df6 = df5.copy()
l = ['6', '2', '9', '5', '3', '1', '4', '7', '8']
l1 = ['11', '10', '1A', '13', '12', '2B', '14', '15', '16', '3X', '1X']
new_d1 = df6[df6["program_number"].isin(l)]
new_d2 = df6[df6["program_number"].isin(l1)]

In [7]:
def insert_two_space(string, integer):
    return string[0:integer] + '  ' + string[integer:]

new_d1['program_number'] = new_d1['program_number'].astype(str).apply(lambda x: insert_two_space(x, 1))

def insert_one_space(string, integer):
    return string[0:integer] + ' ' + string[integer:]

new_d2['program_number'] = new_d2['program_number'].astype(str).apply(lambda x: insert_one_space(x, 2))

/var/folders/yh/rstgp4095fv677pz79rgwyx40000gn/T/ipykernel_17240/1214899009.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_d1['program_number'] = new_d1['program_number'].astype(str).apply(lambda x: insert_two_space(x, 1))
/var/folders/yh/rstgp4095fv677pz79rgwyx40000gn/T/ipykernel_17240/1214899009.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_d2['program_number'] = new_d2['program_number'].astype(str).apply(lambda x: insert_one_space(x, 2))


In [8]:
df7 = pd.concat([new_d1,new_d2])
df7.drop('horse_id', axis=1, inplace=True)

In [9]:
df7['furlongs'] = df7.distance_id / 100
df7.drop('distance_id', axis=1, inplace=True)
df7['new_odds'] = df7['odds'] / 100
df7['new_odds'] = df7['new_odds'].apply(lambda x: round(x,1))
df7['new_odds'] = df7['new_odds'].astype(str)
df7["new_odds"] = df7["new_odds"] + "-1"
df7["implied_probability"] = round(100 / (df7.odds + 100) * 100, 2)
df7.rename(columns={'odds': 'american_odds'}, inplace=True)
df8 = pd.read_csv('nyc_temperature.csv',parse_dates=['date'])
df8.rename(columns={'date': 'race_date'}, inplace=True)
df8.rename(columns={'tavg': 'average_temperature'}, inplace=True)
df8.drop(['tmax', 'tmin','departure','HDD','CDD','precipitation','new_snow','snow_depth'], axis=1, inplace=True)

In [10]:
df9 = pd.merge(df7,df8, on='race_date')

In [11]:
valmap = {
    1: '10', 
    2: '9',
    3: '8',
    4: '7',
    5: '6',
    6: '5',
    7: '4',
    8: '3',
    9: '2',
    10: '1',
    11: '1',
    12: '1',
    13: '1',
    14: '1'
}

df9['rating'] = df9['finishing_place'].map(valmap)

In [12]:
df9['rating'] = df9.rating.astype(int)

In [13]:
df9.to_csv('new_nyra_2019_complete.csv',index=False)
df9.to_parquet('new_nyra_2019_complete.parquet')